# Projeto SiDi - Resolução de Entidade

## Scraper

In [11]:
import requests
import pandas as pd
import re


from bs4 import BeautifulSoup
import time

In [3]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/90.0.4430.212 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

def getdata(url):
    r = requests.get(url, headers = HEADERS)
    return r.text

def html_code(url):
  
    # pass the url
    # into getdata function
    htmldata = getdata(url)
    soup = BeautifulSoup(htmldata, 'html.parser')
  
    # display html code
    return (soup)

In [ ]:
def cus_data(soup):
    # find the Html tag
    # with find()
    # and convert into string

    cus_list= []

    for item in soup.find_all("div", class_="a-section celwidget"):
        review = item.find("span", class_="a-size-base review-text review-text-content").get_text()
        date = item.find("span", class_="a-size-base a-color-secondary review-date").get_text()
        stars = item.find("span", class_="a-icon-alt").get_text()
        
        cus_list.append((review, stars, date))

    return cus_list

In [5]:
url = "https://www.amazon.com.br/s?rh=n%3A16243890011&fs=true&ref=lp_16243890011_sar"
soup = html_code(url)

In [58]:
for link in soup.find_all("div", {"class": "s-main-slot s-result-list s-search-results sg-row"}):
    # display the actual urls
    print(link.get('href'))  

/ref=nav_logo
/gp/css/order-history?ref_=nav_orders_first
/gp/site-directory?ref_=nav_em_js_disabled
/gp/browse.html?node=17877554011&ref_=nav_cs_sell
/gp/bestsellers/?ref_=nav_cs_bestsellers
/prime?ref_=nav_cs_primelink_nonmember
/deals?ref_=nav_cs_gb
/Livros/b/?ie=UTF8&node=6740748011&ref_=nav_cs_books
/music/unlimited?ref_=nav_cs_music
/gp/help/customer/display.html?nodeId=508510&ref_=nav_cs_help
/gp/browse.html?node=17365811011&ref_=nav_cs_fashion
/Eletronicos-e-Tecnologia/b/?ie=UTF8&node=16209062011&ref_=nav_cs_electronics
/Computadores-Informatica/b/?ie=UTF8&node=16339926011&ref_=nav_cs_pc
/gp/new-releases/?ref_=nav_cs_newreleases
/ebooks-kindle/b/?ie=UTF8&node=5475882011&ref_=nav_cs_kindle_books
/Games-Consoles/b/?ie=UTF8&node=7791985011&ref_=nav_cs_video_games
/Ferramentas-Construcao/b/?ie=UTF8&node=16957182011&ref_=nav_cs_hi
/gcx/-/gfhz/?ref_=nav_cs_giftfinder
/Casa/b/?ie=UTF8&node=16191000011&ref_=nav_cs_home
/gp/browse.html?node=16194299011&ref_=nav_cs_toys
/gp/browse.html?n

In [29]:
div_principal = soup.findAll("div", {"class": "s-main-slot s-result-list s-search-results sg-row"})
lis = div_principal.findAll("div", {"class": "a-section a-spacing-none a-spacing-top-small s-title-instructions-style"})


AttributeError: ResultSet object has no attribute 'findAll'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [39]:
results = soup.find_all('div', attrs={"class": "s-main-slot s-result-list s-search-results sg-row"})
records = []  
for result in results:  
    name = result.find('div', attrs={"class": "a-section a-spacing-none a-spacing-top-small s-title-instructions-style"}) # result not results
    records.append(name[])
    
print(records)


from BeautifulSoup import BeautifulSoup
import urllib2

html_page = urllib2.urlopen("https://arstechnica.com")
soup = BeautifulSoup(html_page)
for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
    print link.get('href')

[<div class="a-section a-spacing-none a-spacing-top-small s-title-instructions-style"><h2 class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"><a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/REDMI-NOTE-11-TWILIGHT-128GB/dp/B09QSC4HN9/ref=sr_1_1?qid=1651759712&amp;s=wireless&amp;sr=1-1&amp;ufe=app_do%3Aamzn1.fos.95de73c3-5dda-43a7-bd1f-63af03b14751"><span class="a-size-base-plus a-color-base a-text-normal">REDMI NOTE 11 TWILIGHT BLUE 4GB RAM 128GB ROM</span> </a> </h2></div>]


In [51]:
name['href']

KeyError: 'href'

In [21]:
print(lis)

[<div class="a-section a-spacing-none a-spacing-top-small s-title-instructions-style"><h2 class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"><a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/REDMI-NOTE-11-TWILIGHT-128GB/dp/B09QSC4HN9/ref=sr_1_1?qid=1651759712&amp;s=wireless&amp;sr=1-1&amp;ufe=app_do%3Aamzn1.fos.95de73c3-5dda-43a7-bd1f-63af03b14751"><span class="a-size-base-plus a-color-base a-text-normal">REDMI NOTE 11 TWILIGHT BLUE 4GB RAM 128GB ROM</span> </a> </h2></div>, <div class="a-section a-spacing-none a-spacing-top-small s-title-instructions-style"><h2 class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"><a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/REDMI-NOTE-11-GRAPHITE-128GB/dp/B09QSCYRYH/ref=sr_1_2?qid=1651759712&amp;s=wireless&amp;sr=1-2&amp;ufe=app_do%3Aamzn1.fos.95de73c3-5dda-43a7-bd1f-63af03b14751"><span class="a-size-base-plus a-color-base a-text-

In [ ]:
s-result-list s-search-results sg-row

In [24]:
links = re.findall(r'href=\"(.+?)\"><', str(lis))

51

In [28]:
links[:16]

['/REDMI-NOTE-11-TWILIGHT-128GB/dp/B09QSC4HN9/ref=sr_1_1?qid=1651759712&amp;s=wireless&amp;sr=1-1&amp;ufe=app_do%3Aamzn1.fos.95de73c3-5dda-43a7-bd1f-63af03b14751',
 '/REDMI-NOTE-11-GRAPHITE-128GB/dp/B09QSCYRYH/ref=sr_1_2?qid=1651759712&amp;s=wireless&amp;sr=1-2&amp;ufe=app_do%3Aamzn1.fos.95de73c3-5dda-43a7-bd1f-63af03b14751',
 '/Xiaomi-Redmi-Note-11-Graphite/dp/B09QSB4N2C/ref=sr_1_3?qid=1651759712&amp;s=wireless&amp;sr=1-3&amp;ufe=app_do%3Aamzn1.fos.95de73c3-5dda-43a7-bd1f-63af03b14751',
 '/Smartphone-Redmi-Note-11-128GB/dp/B09QSCYS1J/ref=sr_1_4?qid=1651759712&amp;s=wireless&amp;sr=1-4&amp;ufe=app_do%3Aamzn1.fos.95de73c3-5dda-43a7-bd1f-63af03b14751',
 '/Xiaomi-Redmi-Note-10S-128gb/dp/B093FJZYZ5/ref=sr_1_5?qid=1651759712&amp;s=wireless&amp;sr=1-5&amp;ufe=app_do%3Aamzn1.fos.95de73c3-5dda-43a7-bd1f-63af03b14751',
 '/SMARTPHONE-XIAOMI-REDMI-NOTE-128GB/dp/B098FKC3VX/ref=sr_1_6?qid=1651759712&amp;s=wireless&amp;sr=1-6&amp;ufe=app_do%3Aamzn1.fos.95de73c3-5dda-43a7-bd1f-63af03b14751',
 '/Redmi

In [ ]:
url_inicio = "https://www.amazon.com.br"